In [9]:
using Pkg   
using Distributions,Plots,Parameters,LinearAlgebra,Optim,Roots
@with_kw struct model             
    ## Model parameters:
    β = 0.95 # discount factor
    σ = 2 # RRA/IES
    θ = 0.5 # probability of no information update
    w = 1.0 # fixed riskless income
   
    ## aggregate state:
    nz = 2 # number of agg states
    γgrid = [0.98,1.02] # grid for aggregate state
    p_bg = 0.2 # probability of bad -> good
    p_gg = 0.6 # probability of good -> good
    γprob = [1-p_bg p_bg ; 1-p_gg p_gg] # transition matrix for aggregate state, [x,y] gives probability of transition from x to y

    ## asset grid:
    na = 100 # number of asset grid points
    amin = 0.0 # borrowing constraint
    amax = 1.0 # maximum asset level
    agrid = range(amin,stop=amax,length=na) # asset grid

    ## periods since update grid:
    kmin = 0 # minimum periods since update
    kmax = 20 # θ^kmax should be very small
    nk = 20 # number of periods since update grid points
    kgrid = range(kmin,stop=kmax,length=nk) # periods since update grid

    ## collect all
    stateind = collect(Iterators.product(1:na, 1:nz, 1:nk)) # matrix of state indicies
end

# utility function
function u(par::model,c)
    @unpack σ = par
    if c <= 0
        return -Inf
    else
    if σ == 1
        return log(c)
    else
        return c^(1-σ)/(1-σ)
    end
end;

#include("VFI.jl");

u (generic function with 2 methods)

In [10]:
model().γprob[2,1]

0.4

In [12]:
u(model(),1.0)

-1.0

In [13]:
ttt = zeros(100)

100-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0